In [1]:
import sys
sys.path.append("/mnt/home/TF_NEW/tf-transformers/src/")

In [2]:
!nvidia-smi

Tue Aug 24 16:31:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    43W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   36C    P0    44W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
import tensorflow as tf

from tf_transformers.models import AlbertModel, EncoderDecoder
from transformers import AlbertTokenizerFast

In [4]:
from tf_transformers.core import LegacyModel, LegacyLayer
class Long_Model(LegacyLayer):
    def __init__(
        self, model_layer, num_splits,
        gru_units,
        activation=None, is_training=False, use_dropout=False, **kwargs
    ):
        super(Long_Model, self).__init__(
            is_training=is_training, use_dropout=use_dropout, name=model_layer.name, **kwargs
        )
        self.model_layer = model_layer
        self.num_splits = num_splits
        self.gru_layer = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(gru_units, return_sequences=True,
                                                                           name='gru_for_logits', trainable=True))
        # self.gru_layer_token_embeddings = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(gru_units, return_sequences=True))
        
        self._config_dict = model_layer._config_dict
        self._mask_mode   = model_layer._mask_mode
        self._sequence_length = model_layer._sequence_length
        self.model_inputs, self.model_outputs = self.get_model(initialize_only=True)
    
    def call(self, inputs):
        
        all_outputs_token_embeddings = []
        inputs_splitted = {}
        input_names = []
        for k, v in inputs.items():
            inputs_splitted[k] = tf.split(v, self.num_splits, axis=1)
            input_names.append(k)
            
        for i in range(self.num_splits):
            inputs_main = {}
            for name in input_names:
                inputs_main[name] = inputs_splitted[name][i]
            model_outputs = self.model_layer(inputs_main)
            # all_outputs_token_logits.append(model_outputs["token_logits"])
            all_outputs_token_embeddings.append(model_outputs['token_embeddings'])
            
        # token_logits_concatanted = tf.concat(all_outputs_token_logits, axis=1) # over sequence length

        token_embeddings_concatanted = tf.concat(all_outputs_token_embeddings, axis=1) # over sequence length
        token_embeddings_concatanted = self.gru_layer(token_embeddings_concatanted)
        return {'token_embeddings': token_embeddings_concatanted}
    
 
    def get_model(self, initialize_only=False):
        inputs = {}
        for k, v in self.model_layer.model_inputs.items():
            shape = v.shape
            inputs[k] = tf.keras.layers.Input(
                shape[1:], batch_size=shape[0], name= k, dtype=v.dtype
            )
        layer_output = self(inputs)
        if initialize_only:
            return inputs, layer_output
        model = LegacyModel(inputs=inputs, outputs=layer_output, name="long_span_selection")
        model.model_config = self.model_layer._config_dict
        return model

In [20]:
def get_model():
    gru_units = 384 # half of hidden dimension
    model = AlbertModel.from_pretrained("albert-base-v2", 
                                           return_layer=True)
    long_model = Long_Model(model, num_splits=8, gru_units=gru_units)
    albert_decoder = AlbertModel.from_pretrained("albert-base-v2",use_decoder=True,
                                           return_layer=True, mask_mode='causal')
    
    long_model._layers[0]._embedding_layer = albert_decoder._embedding_layer
    long_model._layers[0]._type_embeddings_layer = albert_decoder._type_embeddings_layer
    long_model._layers[0]._positional_embedding_layer = albert_decoder._positional_embedding_layer
    
    model_encoder = EncoderDecoder(encoder=long_model, decoder=albert_decoder) 
    
    model_encoder = model_encoder.get_model()
    return model_encoder
    

In [6]:
# Convert data to features using specific length
# into a temp dir (and log it as well for monitoring)

def write_tfrecord(data, 
                    batch_size, 
                    tokenizer, 
                    encoder_max_length, 
                    decoder_max_length, 
                    mode, 
                    tfrecord_dir, 
                    take_sample=False, 
                    verbose=10000):
    
    if mode not in ["train", "eval"]:
        raise ValueError("Inavlid mode `{}` specified. Available mode is ['train', 'eval']".format(mode))
    
    def get_tfrecord_example(data):
        for f in data:            
            inputs_hf = tokenizer(f['article'], 
                                  truncation=True, 
                                  max_length=encoder_max_length)

            input_ids  = inputs_hf['input_ids'][:-1] # skip sep
            input_mask = inputs_hf['attention_mask'][:-1] # skip sep
            input_type_ids = [0] * len(input_ids)

            decoder_input_ids = tokenizer(f['abstract'], 
                                  truncation=True, 
                                  max_length=decoder_max_seq_length)['input_ids']

            decoder_input_type_ids = [0] * len(decoder_input_ids)

            result = {}
            result['encoder_input_ids'] = input_ids
            result['encoder_input_mask'] = input_mask
            result['encoder_input_type_ids'] = input_type_ids
            result['decoder_input_ids'] = decoder_input_ids[:-1] # except last word
            result['decoder_input_type_ids'] = decoder_input_type_ids[:-1] # except last word

            result['labels'] = decoder_input_ids[1:] # not including first word
            result['labels_mask'] = [1] * len(result['labels'])

                # Decoder doesnt need input_mask because by default decoder has causal mask mode

            yield result

    schema = {
        "encoder_input_ids": ("var_len", "int"),
        "encoder_input_mask": ("var_len", "int"),
        "encoder_input_type_ids": ("var_len", "int"),
        "decoder_input_ids": ("var_len", "int"),
        "decoder_input_type_ids": ("var_len", "int"),
        "labels": ("var_len", "int"),
        "labels_mask": ("var_len", "int"),
    }
    
    # Create a temp dir
    if mode == "train":
        # Write tf records
        train_data_dir = os.path.join(tfrecord_dir,"train")        
        tfrecord_filename = 'pubmed'
        tfwriter = TFWriter(schema=schema, 
                            file_name=tfrecord_filename, 
                            model_dir=train_data_dir,
                            tag='train',
                            overwrite=True,
                            verbose_counter=verbose
                     )
        data_train = data
        # Take sample
        if take_sample:
            data_train = data_train.select(range(500))
            
        tfwriter.process(parse_fn=get_tfrecord_example(data_train))
    if mode == "eval":
        # Write tfrecords
        eval_data_dir = os.path.join(tfrecord_dir,"eval")
        tfrecord_filename = 'pubmed'
        tfwriter = TFWriter(schema=schema, 
                            file_name=tfrecord_filename, 
                            model_dir=eval_data_dir,
                            tag='eval',
                            overwrite=True,
                            verbose_counter=verbose
                            )
        data_eval = data
        # Take sample
        if take_sample:
            data_eval = data_eval.select(range(500))
        tfwriter.process(parse_fn=get_tfrecord_example(data_eval))
        
def read_tfrecord(tfrecord_dir, max_seq_length, batch_size, shuffle=False, drop_remainder=False):
    
        padded_shapes = {'encoder_input_ids': [max_seq_length,], 
                        'encoder_input_type_ids': [max_seq_length,], 
                        'encoder_input_mask':[max_seq_length,],
                        'decoder_input_ids': [None,],
                        'decoder_input_type_ids': [None,], 
                        'labels': [None,], 
                        'labels_mask': [None,]
                }
        # Read tfrecord to dataset
        schema = json.load(open("{}/schema.json".format(tfrecord_dir)))
        stats  = json.load(open('{}/stats.json'.format(tfrecord_dir)))
        all_files = glob.glob("{}/*.tfrecord".format(tfrecord_dir))
        tf_reader = TFReader(schema=schema, 
                            tfrecord_files=all_files)

        x_keys = ['encoder_input_ids', 'encoder_input_type_ids', 'encoder_input_mask', 'decoder_input_ids', 'decoder_input_type_ids']
        y_keys = ['labels', 'labels_mask']
        dataset = tf_reader.read_record(auto_batch=True, 
                                           keys=x_keys,
                                           padded_shapes=padded_shapes,
                                           batch_size=batch_size, 
                                           x_keys = x_keys, 
                                           y_keys = y_keys,
                                           shuffle=shuffle, 
                                           drop_remainder=drop_remainder
                                          )
        return dataset, stats['total_records']

In [7]:

# Data specific configuration
encoder_max_seq_length = 4096
decoder_max_seq_length = 256

take_sample = False
train_batch_size = 4
eval_batch_size  = 8

# Trainer specifics
device = "gpu"
num_gpus = 2
tpu_address = None
dtype = "fp32"
epochs = 3
strategy = "mirrored"

# Optimizer
learning_rate = 3e-5
loss_type = None
return_all_layer_outputs = False
if loss_type and loss_type == 'joint':
    return_all_layer_outputs = True

# Core data specifics
data_name = "scientific_papers"
#num_classes = cfg.glue.data.num_classes

# Model specific
is_training = False
use_dropout = False

In [10]:
import glob
import json
from tf_transformers.data import TFWriter, TFReader
tokenizer = AlbertTokenizerFast.from_pretrained("albert-base-v2")

In [11]:
# Load dataset
from datasets import load_dataset
dataset = load_dataset("scientific_papers", "pubmed")

Reusing dataset scientific_papers (/home/jovyan/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


In [12]:
# Load TFrecords
# tfrecord_dir = tempfile.mkdtemp()
tfrecord_dir = '/tmp/roberta2robera_pubmed_long_tfrecords/'

# # Train Tfrecords
# write_tfrecord(dataset['train'], 
#                train_batch_size,
#                tokenizer, 
#                encoder_max_seq_length, 
#                decoder_max_seq_length, 
#                "train", 
#                tfrecord_dir, 
#                take_sample, 
#                verbose=10000)

# # # Eval Tfrecords
# write_tfrecord(dataset['validation'], 
#                eval_batch_size,
#                tokenizer, 
#                encoder_max_seq_length, 
#                decoder_max_seq_length, 
#                "eval", 
#                tfrecord_dir, 
#                take_sample, 
#               verbose=10000)



train_dataset, total_train_examples = read_tfrecord(tfrecord_dir + 'train', encoder_max_seq_length, train_batch_size, shuffle=True, drop_remainder=True)
eval_dataset, total_eval_examples   = read_tfrecord(tfrecord_dir + 'eval', encoder_max_seq_length,  eval_batch_size,  shuffle=False, drop_remainder=False)

# original_summaries = [item['summary'] for item in dataset['validation']]
# callback = RougeCallback( model_ar, eval_dataset, original_summaries,
#                          tokenizer, tokenizer.sep_token_id, tokenizer.cls_token_id, decoder_max_seq_length)

In [13]:
for (batch_inputs, batch_labels) in train_dataset:
    print(batch_inputs, batch_labels)
    break

{'decoder_input_ids': <tf.Tensor: shape=(4, 255), dtype=int32, numpy=
array([[    2, 24122,  3220, ...,     0,     0,     0],
       [    2,    19,  1764, ...,  6522,  1660,  7426],
       [    2,    13,  9571, ...,     0,     0,     0],
       [    2,   302, 17083, ...,    50,   166,    91]], dtype=int32)>, 'decoder_input_type_ids': <tf.Tensor: shape=(4, 255), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'encoder_input_ids': <tf.Tensor: shape=(4, 4096), dtype=int32, numpy=
array([[    2, 14360,  4776, ...,  5477,    13,     0],
       [    2,    14,   275, ...,     0,     0,     0],
       [    2, 18436,    17, ...,     0,     0,     0],
       [    2,   302, 17083, ...,     0,     0,     0]], dtype=int32)>, 'encoder_input_mask': <tf.Tensor: shape=(4, 4096), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0,

In [14]:
# Load optimizer fn

from tf_transformers.optimization import create_optimizer
def get_optimizer(learning_rate, examples, batch_size, epochs, decay_function="polynomial"):
    steps_per_epoch = int(examples / batch_size)
    num_train_steps = steps_per_epoch * epochs
    warmup_steps = int(0.1 * num_train_steps)
    
    def optimizer_fn():
        optimizer, learning_rate_fn = create_optimizer(learning_rate,
                                                   num_train_steps,
                                                   num_train_steps, 
                                                      decay_function=decay_function)
        return optimizer
    return optimizer_fn

learning_rate =  3e-5
optimizer_fn = get_optimizer(learning_rate, total_train_examples, train_batch_size, epochs, decay_function="linear")

In [15]:
from tf_transformers.losses import cross_entropy_loss, cross_entropy_loss_label_smoothing

# Loss fn
def get_loss(y_true_dict, y_pred_dict):
    
    loss = cross_entropy_loss_label_smoothing(labels=y_true_dict['labels'], 
                                   logits=y_pred_dict['token_logits'], 
                                      label_weights=y_true_dict['labels_mask'])
    return {'loss': loss}

In [16]:
# Load trainer
from tf_transformers.core import GPUTrainer
strategy = 'mirrored'
num_gpus = 2
dtype = 'fp32'
trainer = GPUTrainer(distribution_strategy=strategy, 
                    num_gpus=num_gpus, 
                    dtype=dtype)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [17]:
epochs = 3

In [18]:
total_train_examples//train_batch_size

29981

In [ ]:
model_checkpoint_dir = "/tmp/roberta2robera_pubmed_long_sequence/"
history = trainer.run(
    model_fn = get_model,
    optimizer_fn = optimizer_fn,
    train_dataset = train_dataset,
    train_loss_fn = get_loss,
    epochs = epochs,
    steps_per_epoch = 29981,
    model_checkpoint_dir=model_checkpoint_dir,
    batch_size=train_batch_size,
    training_loss_names=None,
    validation_loss_names=None,
    validation_dataset=None,
    validation_loss_fn=None,
    validation_interval_steps=None,
    steps_per_call=1,
    enable_xla=False,
    callbacks=None,
    callbacks_interval_steps=None,
    overwrite_checkpoint_dir=True,
    max_number_of_models=10,
    model_save_interval_steps=None,
    repeat_dataset=False,
    latest_checkpoint=None,
)

INFO:absl:Make sure `steps_per_epoch` should be less than or equal to number of batches in dataset.
INFO:absl:Policy: ----> float32
INFO:absl:Strategy: ---> <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fd234021730>
INFO:absl:Num GPU Devices: ---> 2
You are using a model of type albert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
INFO:absl:Successful: Model checkpoints matched and loaded from /tmp/tf_transformers_cache/albert-base-v2/ckpt-1
You are using a model of type albert to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
INFO:absl:Successful: Model checkpoints matched and loaded from /tmp/tf_transformers_cache/albert-base-v2/ckpt-1
INFO:absl:Using Adamw optimizer
INFO:absl:No checkpoint found in /tmp/roberta2robera_pubmed_long_sequence/
Epoch 1/3 --- Step 1/29981 --- total examples 0:   0%|          | 0/29981 [00:00<?, ?batch 

INFO:tensorflow:batch_all_reduce: 63 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 63 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


...

INFO:tensorflow:batch_all_reduce: 63 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/3 --- Step 21/29981 --- total examples 80:   0%|          | 20/29981 [05:46<78:00:12,  9.37s/batch , learning_rate=6.67e-9, loss=7.02] 